In [1]:
pip install torch

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
import numpy as np

# Sample data (use your training dataset here)
data = [("Hello", "Hi there!"), ("How are you?", "I'm doing well, thank you!")]

# Build vocabulary
class Vocabulary:
    def __init__(self):
        self.word2index = {}
        self.index2word = {}
        self.index = 0

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.index
            self.index2word[self.index] = word
            self.index += 1

    def add_sentence(self, sentence):
        for word in sentence.split():
            self.add_word(word)

vocab = Vocabulary()
for pair in data:
    vocab.add_sentence(pair[0])
    vocab.add_sentence(pair[1])

# Tokenize a sentence
def tokenize(sentence):
    return [vocab.word2index[word] for word in sentence.split() if word in vocab.word2index]

In [3]:
class ChatDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text, target_text = self.data[idx]
        input_tokens = tokenize(input_text)
        target_tokens = tokenize(target_text)
        return torch.tensor(input_tokens), torch.tensor(target_tokens)

dataset = ChatDataset(data, vocab)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [4]:
class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SimpleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (hidden, cell) = self.lstm(embedded)
        output = self.fc(lstm_out[:, -1, :])  # Take the last LSTM output
        return output

# Model parameters
embedding_dim = 16
hidden_dim = 32
output_dim = len(vocab.word2index)

model = SimpleLSTM(len(vocab.word2index), embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [5]:
num_epochs = 100

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets.squeeze())  # Targets are expected to be 1D for CrossEntropyLoss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


ValueError: Expected input batch_size (1) to match target batch_size (2).

In [ ]:
def generate_response(input_text):
    input_tokens = torch.tensor(tokenize(input_text)).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(input_tokens)
        predicted_index = torch.argmax(output, dim=1).item()
        response_word = vocab.index2word[predicted_index]
    return response_word

# Example inference
user_input = "Hello"
print("Bot response:", generate_response(user_input))
